In [ ]:
%pip install geopandas geoplot shapely bokeh numpy geopy folium pandas

In [21]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import pandas as pd

# The excel file location within google drive (e.g. /content/gdrive/my_data.xlsx)
# THIS WILL NEED TO BE CHANGED BASED ON YOUR DATA FILE NAME
basepath = "/content/gdrive/MyDrive/"
filename = basepath + "cities_locations.xlsx"
filename = "L:\Equity Research\REIT Team Shared\Residential (New)\MF\Seattle Practice Supply.xlsx"

# Read the entirety of the excel workbook with the following structure
"""

each sheet has the following columns:
  Existing_lat | existing_lon | construction_lat | construction_lon | num units

each sheet is named by the city where the properties in these columns are located
(note this is not strictly necessary but does help organize things more easily)
"""

lat_lon_city_df_dict = pd.read_excel(filename, sheet_name=None)

In [2]:
# column headers
for city, city_df in lat_lon_city_df_dict.items():
  print(f"city {city}")
  print(city_df.head())

city Sheet1
        ELat        ELon       CLat        CLon  num units
0  37.282850 -121.938163  47.615017 -122.309073       25.0
1  37.333017 -122.021493  47.608640 -122.316632       26.0
2  37.414114 -121.903727  47.605856 -122.313105       27.0
3  37.397323 -122.092631  47.611858 -122.190724       28.0
4  37.336272 -121.887088  47.679610 -122.164069       29.0


In [3]:
nyc_df = lat_lon_city_df_dict['New York']

nyc_existing = nyc_df[['ELat', 'ELon']]
nyc_construction_units = nyc_df[['CLat', 'CLon', 'num units']]

print(nyc_existing)
print(nyc_construction_units)
print(type(nyc_existing['ELat'][0]))

KeyError: 'New York'

In [4]:
def get_distance_between_points(lat1, lon1, lat2, lon2, threshold=1, unit='mi'):
  """
    calculate the distance in 'units' of the input coordinates given by 'lat1/2' and lon1/2'.
    If the resulting distance falls within 'threshold' then we are going to increment the
    encroach count by the number of units of the building under construction. If the units have already
    been accounted for, then the sum only increments by 1 (is correct? why?)
  """
  assert unit in ['mi', 'km'], f"unit {unit} is not valid. Must be mi or km"

  # calculate the distance between 2 coordinate pairs (lat N, lon W) in specified units
  # determine whether the resulting distance violates the specified threshold

  import geopy.distance
  
  coords1 = (lat1, lon1) # tuple type
  coords2 = (lat2, lon2) # tuple type
  
  if unit == 'mi':
    distance = geopy.distance.geodesic(coords1, coords2).miles
  elif unit == 'km':
    distance = geopy.distance.geodesic(coords1, coords2).km

  print(f"distance between {coords1} and {coords2} in {unit} is {distance} {unit}")

  return distance

In [ ]:
threshold = 1 # miles

violation_city_df_dict = {}
total_units_interfering = {}

companies = ["AVB", "ESS", "EQR", "UDR", "CPT", "MAA"]

# for company in companies:
# AVB, ESS, EQR, UDR, CPT, MAA - sheet names by company
for city,df in lat_lon_city_df_dict.items():

  #df.dropna(inplace=True)

  existing = df[['ELat','ELon']]

  construction = df[['CLat', 'CLon', 'num units']]
  construction.dropna (inplace = True)

  construction_sites_did_count = {(row['CLat'], row['CLon']): [False, row['num units']] for _, row in construction.iterrows()}
  
  total_units_interfering[city] = 0
  
  print(construction_sites_did_count)

  violation_df = pd.DataFrame()

  num_violations = 0

  clat_vlst = [] # for col CLat
  clon_vlst = [] # for col CLon
  elat_vlst = [] # for col ELat
  elon_vlst = [] # for col ELon
  dist_vlst = [] # for col interf_dist_mi
  unit_vlst = [] # for col num_units_interfering

  for index, row in construction.iterrows():
    
    clat = row['CLat']
    clon = row['CLon']
    unit_count = row['num units']

    for index, row in existing.iterrows():
      elat = row['ELat']
      elon = row['ELon']

      # get the distance
      distance = get_distance_between_points(clat, clon, elat, elon)

      if distance <= threshold and construction_sites_did_count[(clat, clon)][0] == False:
        print(f"[violation: not counted] construction at ({clat}, {clon}) interferes with building at ({elat},{elon}) with distance {distance} mi")

        # mark construction site as visited
        construction_sites_did_count[(clat, clon)][0] = True
        interfering_units = construction_sites_did_count[(clat, clon)][1]
        total_units_interfering[city] += interfering_units

        # add the interference to the lists to build the dataframe
        clat_vlst.append(clat)
        clon_vlst.append(clon)
        elat_vlst.append(elat)
        elon_vlst.append(elon)
        dist_vlst.append(distance)
        unit_vlst.append(interfering_units)

        # update total number of violations
        num_violations += 1

      elif distance <= threshold and construction_sites_did_count[(clat, clon)][0] == True:
        print(f"[violation: already counted] already counted construction at ({clat}, {clon}) which interferes with building at ({elat},{elon}) with distance {distance} mi")
      
      else:
        print(f"no violation between construction at ({clat}, {clon}) and existing building ({elat},{elon}) (distance: {distance} mi")

      print("*" * 30)
    # update the dataframe and write to dict
  print(len(clat_vlst))
  violation_df['CLat'] = clat_vlst
  violation_df['CLon'] = clon_vlst
  violation_df['ELat'] = elat_vlst
  violation_df['ELon'] = elon_vlst
  violation_df['interf_dist_mi'] = dist_vlst
  violation_df['num_units_interfering'] = unit_vlst

  violation_city_df_dict[city] = violation_df


print(construction_sites_did_count)
print(total_units_interfering)
print(violation_city_df_dict)

In [8]:
import folium
from geopy.geocoders import Nominatim

In [9]:
def get_coords_by_cityname(city):
  geolocator = Nominatim(user_agent='myapplication')
  location = geolocator.geocode(city)
  print(location.raw)
  return (location.raw['lat'], location.raw['lon'])

In [12]:
def create_map(city_name, city_interference_df_dict, circles=True, threshold=1, units='mi', construction_marker_color='red', existing_marker_color='blue'):

  city_interference_df = city_interference_df_dict[city_name]

  # 100 mi circle around average coordinate to get (# interfering units / total unit count under construction)

  #city_coords = get_coords_by_cityname(city_name)
  #city_lat, city_lon = city_coords
  #print(f"creating map for city: {city_name} at coords N lat, W lon ({city_lat}, {city_lon})")

  m = folium.Map(zoom_start=12, tiles='OpenStreetMap', csr="EPSG4326")

  # 40.7128, 74.0060

  #folium.Circle(
          #location= [city_lat, city_lon],
          #radius=1609 * 25,
          #popup= f"{threshold} {units} radius",
          #color='green',
          #fill=False,
      #).add_to(m)

  for index, row in city_interference_df.iterrows():

    # plot the existing site
    clat = row['CLat']
    clon = row['CLon']
    print(f"clat, clon: {round(clat, 3)}, {round(clon, 3)}, {type(clat), type(clon)}")
    unit_count = row['num_units_interfering']

    folium.Marker(
      location= [round(clat, 3), round(clon, 3)],
      radius=3,
      #color=construction_marker_color,
      fill=True,
      icon=folium.Icon(color=construction_marker_color, icon=''),
      popup= f"construction location ({clat},{clon})"   # f"{unit_count} units in progress",
    ).add_to(m)
    
    elat = row['ELat']
    elon = row['ELon']

    if circles:
      # plot 1mi radius around location
      folium.Circle(
          location= [elat, elon],
          radius=1609,
          popup= f"{threshold} {units} radius",
          color=existing_marker_color,
          fill=False,
      ).add_to(m)

    #plot existing location
    folium.Marker(
        location= [elat, elon],
        radius=3,
        color=existing_marker_color,
        fill=True,
        fill_color=existing_marker_color,
        popup=f"existing location ({elat}, {elon})"
    ).add_to(m)

    dist = row['interf_dist_mi']

  return m

In [14]:
for city in list(violation_city_df_dict.keys()):
  print("=" * 40)
  m = create_map(city, violation_city_df_dict)

m

clat, clon: 47.612, -122.191, (<class 'numpy.float64'>, <class 'numpy.float64'>)
